In [1]:
import ast
import glob
import warnings
from collections import defaultdict
from datetime import date

import numpy as np
import pandas as pd
import wandb

today = date.today()
api = wandb.Api()

# # Find all csv files in the current directory
csv_files = glob.glob("*.csv")
# # Collect all the names of the csv files without the extension
csv_names = [csv_file[:-4] for csv_file in csv_files]
project_name = "TopoBench"  #'best_results_edhnn'
user = "telyatnikov_sap"

if project_name not in csv_names:
    runs = api.runs(f"{user}/{project_name}")

    summary_list, config_list, name_list = [], [], []
    for run in runs:
        # .summary contains the output keys/values for metrics like accuracy.
        #  We call ._json_dict to omit large files
        summary_list.append(run.summary._json_dict)

        # .config contains the hyperparameters.
        #  We remove special values that start with _.
        config_list.append(
            {k: v for k, v in run.config.items() if not k.startswith("_")}
        )

        # .name is the human-readable name of the run.
        name_list.append(run.name)

    runs_df = pd.DataFrame(
        {"summary": summary_list, "config": config_list, "name": name_list}
    )

    runs_df.to_csv(f"{project_name}.csv")
else:
    runs_df = pd.read_csv(f"{project_name}.csv", index_col=0)

    for row in runs_df.iloc:
        row["summary"] = ast.literal_eval(row["summary"])
        row["config"] = ast.literal_eval(row["config"])


for row in runs_df.iloc:
    row["summary"].update(row["config"])

lst = [i["summary"] for i in runs_df.iloc]
df = pd.DataFrame.from_dict(lst)

df_init = df.copy()

# Get average epoch run time
df["epoch_run_time"] = df["_runtime"] / df["epoch"]

In [ ]:
import wandb

api = wandb.Api()
run = api.run(f"{user}/{project_name}/run_id")

# List all files attached to this run
for file in run.files():
    print(file.name)

# Download a specific log file
file = run.file('output.log')
file.download(replace=True) 

In [115]:
import wandb
import os

# --- CONFIGURATION ---
project_name = "TopoBench"
entity = "telyatnikov_sap"
output_dir = "downloaded_logs"

# --- CONNECT TO WANDB API ---
api = wandb.Api()
runs = api.runs(f"{entity}/{project_name}")

# Select the first run (or change index as needed)
run = runs[0]
run_id = run.id
run_name = run.name

print(f"Downloading logs for run: {run_name} (ID: {run_id})")

# Create local directory for saving logs
os.makedirs(output_dir, exist_ok=True)
run_dir = os.path.join(output_dir, f"{run_name}_{run_id}")
os.makedirs(run_dir, exist_ok=True)

# List and download all log files
for file in run.files():
    if file.name.endswith('.log') or 'logs' in file.name:
        print(f"⬇ Downloading {file.name}")
        file.download(root=run_dir, replace=True)

print(f"\n✅ All log files saved to: {run_dir}")



⬇ Downloading output.log

✅ All log files saved to: downloaded_logs/fast-jazz-1_fsnrbsfs


/tmp/ipykernel_1136837/3568717637.py:29: ResourceWarning: unclosed file <_io.TextIOWrapper name='downloaded_logs/fast-jazz-1_fsnrbsfs/output.log' mode='r' encoding='UTF-8'>
  file.download(root=run_dir, replace=True)


In [121]:
list(run.files())

[<File artifact/1611705586/wandb_manifest.json (application/json) 0.0B>,
 <File config.yaml () 10.7KiB>,
 <File output.log () 11.4KiB>,
 <File requirements.txt (text/plain; charset=utf-8) 5.1KiB>,
 <File wandb-summary.json (application/json) 1017.0B>]

In [2]:
df_init.columns

Index(['AvgTime/train_batch_mean', 'AvgTime/train_batch_std',
       'AvgTime/train_epoch_mean', 'AvgTime/train_epoch_std',
       'AvgTime/val_batch_mean', 'AvgTime/val_batch_std',
       'AvgTime/val_epoch_mean', 'AvgTime/val_epoch_std', '_runtime', '_step',
       '_timestamp', '_wandb', 'epoch', 'lr-Adam', 'test/accuracy',
       'test/auroc', 'test/loss', 'test/precision', 'test/recall',
       'train/accuracy', 'train/auroc', 'train/loss', 'train/precision',
       'train/recall', 'trainer/global_step', 'val/accuracy', 'val/auroc',
       'val/loss', 'val/precision', 'val/recall', 'loss', 'seed', 'tags',
       'test', 'model', 'paths', 'train', 'extras', 'logger', 'dataset',
       'trainer', 'callbacks', 'ckpt_path', 'evaluator', 'optimizer',
       'task_name', 'transforms', 'model/params/total',
       'model/params/trainable', 'model/params/non_trainable', 'test/mae',
       'test/mse', 'train/mae', 'train/mse', 'val/mae', 'val/mse'],
      dtype='object')

In [3]:
def normalize_column(df, column_to_normalize):
    # Use json_normalize to flatten the nested dictionaries into separate columns
    flattened_df = pd.json_normalize(df[column_to_normalize])
    # Rename columns to include 'nested_column' prefix
    flattened_df.columns = [
        f"{column_to_normalize}.{col}" for col in flattened_df.columns
    ]
    # Concatenate the flattened DataFrame with the original DataFrame
    result_df = pd.concat([df, flattened_df], axis=1)
    # Get new columns names
    new_columns = flattened_df.columns
    # Drop the original nested column if needed
    result_df.drop(column_to_normalize, axis=1, inplace=True)
    return result_df, new_columns


# Config columns to normalize
columns_to_normalize = ["model", "dataset", "callbacks", "paths"]

# Keep track of config columns added
config_columns = []
for column in columns_to_normalize:
    df, columns = normalize_column(df, column)
    config_columns.extend(columns)

In [4]:
idx=df['dataset.loader.parameters.data_name'] =='US-county-demos'
df.loc[idx,'dataset.loader.parameters.data_name'] = df.loc[idx,'dataset.loader.parameters.task_variable']

In [5]:
# Remove columns that are not needed (we shouldn't vary them or their variation is not interesting)
remove_col = [
    #"dataset.transforms.data_manipulations.selected_fields",
    "callbacks.model_checkpoint.dirpath", 'model.feature_encoder.selected_dimensions','callbacks.model_checkpoint.dirpath',
]
df = df.drop(remove_col, axis=1)

# Ensure that removed columns are not present in config_columns
config_columns = [col for col in config_columns if col != remove_col]

In [6]:
print(
    f"Number of rows with model.backbone._target_ = nan is {sum(df['model.backbone._target_'].isna())}"
)
# Drop na values if there are
df = df.dropna(subset=["model.backbone._target_"])
# Reset index
df = df.reset_index(drop=True)

# Drop rows that 'callbacks.early_stopping.monitor' isna
print(
    f"Number of rows with callbacks.early_stopping.monitor = nan is {sum(df['callbacks.early_stopping.monitor'].isna())}"
)

# print("Because of SCCN and CWN false runs there were 96 such runs on 13/03/24")

df = df.dropna(subset=["callbacks.early_stopping.monitor"])
# Reset index
df = df.reset_index(drop=True)


# Get correct names for the models
df["model.backbone._target_"] = df["model.backbone._target_"].apply(
    lambda x: x.split(".")[-1]
)

Number of rows with model.backbone._target_ = nan is 0
Number of rows with callbacks.early_stopping.monitor = nan is 0


In [7]:
df["model.backbone._target_"].unique()

array(['CCCN', 'CCXN', 'CWN', 'GAT', 'GCN', 'GIN', 'AllSetTransformer',
       'EDGNN', 'UniGCNII', 'SCCN', 'SCCNNCustom', 'SCN2'], dtype=object)

In [8]:
# Identify unique models in DataFrame
unique_models = df["model.backbone._target_"].unique()

# Identify unique datasets in DataFrame
unique_datasets = df["dataset.loader.parameters.data_name"].unique()


collected_results = defaultdict(dict)
collected_results_time = defaultdict(dict)
collected_results_time_run = defaultdict(dict)

collected_aggregated_results = defaultdict(dict)
collected_non_aggregated_results = defaultdict(dict)

# Got over each dataset and model and find the best result
for dataset in unique_datasets:
    for model in unique_models:
        # Get the subset of the DataFrame for the current dataset and model
        subset = df[
            (df["dataset.loader.parameters.data_name"] == dataset)
            & (df["model.backbone._target_"] == model)
        ]

        if subset.empty:
            print("---------")
            print(f"No results for {model} on {dataset}")
            print("---------")
            continue
        # Suppress all warnings
        warnings.filterwarnings("ignore")
        subset["Model"] = model
        warnings.filterwarnings("default")

        def get_metric(df):
            metric_ = df["callbacks.early_stopping.monitor"].unique()
            assert len(metric_) == 1, "There should be only one metric to optimize"
            metric = metric_[0]
            return metric.split("/")[-1]

        # Cols to get statistics later
        # TODO: log maximum validation value for optimized metric
        performance_cols = [f"test/{get_metric(subset)}"]

        # Get the unique values for each config column
        unique_colums_values = {}
        for col in config_columns:
            try:
                unique_colums_values[col] = subset[col].unique()
            except:
                print(f"Attention the columns: {col}, has issues with unique values")

        # Keep only those keys that have more than one unique value
        unique_colums_values = {
            k: v for k, v in unique_colums_values.items() if len(v) > 1
        }

        # Print the unique values for each config column

        print(f"Unique values for each config column for {model} on {dataset}:")
        for col, unique in unique_colums_values.items():
            print(f"{col}: {unique}")
            print()
        print("---------")

        # Check if "special colums" are not in unique_colums_values
        # For example dataset.parameters.data_seed should not be in aggregation columns
        # If it is, then we should remove it from the list
        special_columns = ["dataset.parameters.data_seed"]

        for col in special_columns:
            if col in unique_colums_values:
                unique_colums_values.pop(col)

        # Obtain the aggregation columns
        aggregation_columns = ["Model"] + list(unique_colums_values.keys())

        collected_non_aggregated_results[dataset][model] = {
            "df": subset.copy(),
            "aggregation_columns": aggregation_columns,
            "performance_cols": performance_cols,
        }

        # Aggregate the subset by the aggregation columns and get the best result for each group
        aggregated = subset.groupby(aggregation_columns).agg(
            {col: ["mean", "std"] for col in performance_cols}
        )

        # Go from MultiIndex to Index
        aggregated = aggregated.reset_index()

        assert (
            len(subset["callbacks.early_stopping.mode"].unique()) == 1
        ), "There should be only one mode for early stopping"
        # Identify the mode of the early stopping mode

        if subset["callbacks.early_stopping.mode"].unique()[0] == "max":
            ascending = False
            final_best_ = aggregated.sort_values(
                by=(f"test/{get_metric(subset)}", "mean"), ascending=ascending
            ).head(1)
            final_best_ = (final_best_ * 100).round(2)
        else:
            ascending = True
            final_best_ = aggregated.sort_values(
                by=(f"test/{get_metric(subset)}", "mean"), ascending=ascending
            ).head(1)

        collected_results[dataset][model] = {
            "mean": final_best_[(f"test/{get_metric(subset)}", "mean")].values[0],
            "std": final_best_[(f"test/{get_metric(subset)}", "std")].values[0],
        }

        # Get average epoch run time
        collected_results_time[dataset][model] = {
            "mean": subset['AvgTime/train_epoch_mean'].mean(),
            "std": subset['AvgTime/train_epoch_mean'].std(),
        }

        collected_results_time_run[dataset][model] = {
            "mean": subset["_runtime"].mean(),
            "std": subset["_runtime"].std(),
        }

        collected_aggregated_results[dataset][model] = aggregated.sort_values(
            by=(f"test/{get_metric(subset)}", "mean"), ascending=ascending
        )

Attention the columns: model.feature_encoder.in_channels, has issues with unique values
Attention the columns: model.feature_encoder.selected_dimensions, has issues with unique values
Attention the columns: callbacks.model_checkpoint.dirpath, has issues with unique values
Unique values for each config column for CCCN on IMDB-BINARY:
dataset.split_params.data_seed: [0 3 5 7 9]

paths.output_dir: ['/home/levtel/projects/TopoBench/logs/train/multiruns/2025-03-17_03-08-10/0'
 '/home/levtel/projects/TopoBench/logs/train/multiruns/2025-03-17_03-08-10/1'
 '/home/levtel/projects/TopoBench/logs/train/multiruns/2025-03-17_03-08-10/2'
 '/home/levtel/projects/TopoBench/logs/train/multiruns/2025-03-17_03-08-10/3'
 '/home/levtel/projects/TopoBench/logs/train/multiruns/2025-03-17_03-08-10/4']

---------
Attention the columns: model.feature_encoder.in_channels, has issues with unique values
Attention the columns: model.feature_encoder.selected_dimensions, has issues with unique values
Attention the co

Attention the columns: model.feature_encoder.in_channels, has issues with unique values
Attention the columns: model.feature_encoder.selected_dimensions, has issues with unique values
Attention the columns: callbacks.model_checkpoint.dirpath, has issues with unique values
Unique values for each config column for CCCN on NCI109:
dataset.split_params.data_seed: [0 3 5 7 9]

paths.output_dir: ['/home/levtel/projects/TopoBench/logs/train/multiruns/2025-03-17_03-54-21/0'
 '/home/levtel/projects/TopoBench/logs/train/multiruns/2025-03-17_03-54-21/1'
 '/home/levtel/projects/TopoBench/logs/train/multiruns/2025-03-17_03-54-21/2'
 '/home/levtel/projects/TopoBench/logs/train/multiruns/2025-03-17_03-54-21/3'
 '/home/levtel/projects/TopoBench/logs/train/multiruns/2025-03-17_03-54-21/4']

---------
Attention the columns: model.feature_encoder.in_channels, has issues with unique values
Attention the columns: model.feature_encoder.selected_dimensions, has issues with unique values
Attention the columns

### Runtime per epoch

In [14]:
# List from Screenshot 1
list1_data = [
    "Cora",
    "Citeseer",
    "Pubmed",
    "MUTAG",
    "PROTEINS",
    "NCI1",
    "NCI109",
    "IMDB-BIN",
    "IMDB-MUL",
    "REDDIT",
    "Amazon",
    "Minesweeper",
    "Empire",
    "Tolokers",
    "Election",
    "Bachelor",
    "Birth",
    "Death",
    "Income",
    "Migration",
    "Unempl",
    "ZINC"
]

# List from Screenshot 2
list2_data = [
    "BachelorRate",
    "BirthRate",
    "Cora",
    "DeathRate",
    "Election",
    "IMDB-BINARY",
    "IMDB-MULTI",
    "MUTAG",
    "MedianIncome",
    "MigraRate",
    "NCI1",
    "NCI109",
    "PROTEINS",
    "PubMed",
    "REDDIT-BINARY",
    "UnemploymentRate",
    "ZINC",
    "amazon_ratings",
    "citeseer",
    "minesweeper",
    "roman_empire"
]

# Dictionary mapping from list2 to list1
mapping_dict_data = {
    "BachelorRate": "Bachelor",
    "BirthRate": "Birth",
    "Cora": "Cora",
    "DeathRate": "Death",
    "Election": "Election",
    "IMDB-BINARY": "IMDB-BIN",
    "IMDB-MULTI": "IMDB-MUL",
    "MUTAG": "MUTAG",
    "MedianIncome": "Income",
    "MigraRate": "Migration",
    "NCI1": "NCI1",
    "NCI109": "NCI109",
    "PROTEINS": "PROTEINS",
    "PubMed": "Pubmed",
    "REDDIT-BINARY": "REDDIT",
    "UnemploymentRate": "Unempl",
    "ZINC": "ZINC",
    "amazon_ratings": "Amazon",
    "citeseer": "Citeseer",
    "minesweeper": "Minesweeper",
    "roman_empire": "Empire",
    "tolokers": "Tolokers",
}

print("List 1 (Screenshot 1):")
print(list1_data)
print("\nList 2 (Screenshot 2):")
print(list2_data)
print("\nMapping Dictionary (List 2 -> List 1):")
print(mapping_dict_data)





List 1 (Screenshot 1):
['Cora', 'Citeseer', 'Pubmed', 'MUTAG', 'PROTEINS', 'NCI1', 'NCI109', 'IMDB-BIN', 'IMDB-MUL', 'REDDIT', 'Amazon', 'Minesweeper', 'Empire', 'Tolokers', 'Election', 'Bachelor', 'Birth', 'Death', 'Income', 'Migration', 'Unempl', 'ZINC']

List 2 (Screenshot 2):
['BachelorRate', 'BirthRate', 'Cora', 'DeathRate', 'Election', 'IMDB-BINARY', 'IMDB-MULTI', 'MUTAG', 'MedianIncome', 'MigraRate', 'NCI1', 'NCI109', 'PROTEINS', 'PubMed', 'REDDIT-BINARY', 'UnemploymentRate', 'ZINC', 'amazon_ratings', 'citeseer', 'minesweeper', 'roman_empire']

Mapping Dictionary (List 2 -> List 1):
{'BachelorRate': 'Bachelor', 'BirthRate': 'Birth', 'Cora': 'Cora', 'DeathRate': 'Death', 'Election': 'Election', 'IMDB-BINARY': 'IMDB-BIN', 'IMDB-MULTI': 'IMDB-MUL', 'MUTAG': 'MUTAG', 'MedianIncome': 'Income', 'MigraRate': 'Migration', 'NCI1': 'NCI1', 'NCI109': 'NCI109', 'PROTEINS': 'PROTEINS', 'PubMed': 'Pubmed', 'REDDIT-BINARY': 'REDDIT', 'UnemploymentRate': 'Unempl', 'ZINC': 'ZINC', 'amazon_rating

In [15]:
# List from Screenshot (List 1)
list1_model = [
    "GCN",
    "GAT", 
    "GIN",
    "AST",
    "EDGNN",
    "UniGNN2",
    "CWN",
    "CCCN",
    "CCXN",
    "SCN",
    "SCCN",
    "SCCNN"
]

# New List 2
list2_model = [
    "AllSetTransformer",
    "CCCN",
    "CCXN", 
    "CWN",
    "EDGNN",
    "GAT",
    "GCN",
    "GIN",
    "SCCN",
    "SCCNNCustom",
    "SCN2",
    "UniGCNII"
]

# Dictionary mapping from list2 to list1
mapping_dict_model = {
    "AllSetTransformer": "AST",  # AllSetTransformer -> AST
    "CCCN": "CCCN",  # Exact match
    "CCXN": "CCXN",  # Exact match
    "CWN": "CWN",    # Exact match
    "EDGNN": "EDGNN",  # Exact match
    "GAT": "GAT",    # Exact match
    "GCN": "GCN",    # Exact match
    "GIN": "GIN",    # Exact match
    "SCCN": "SCCN",  # Exact match
    "SCCNNCustom": "SCCNN",  # SCCNNCustom -> SCCNN
    "SCN2": "SCN",   # SCN2 -> SCN
    "UniGCNII": "UniGNN2"  # UniGCNII -> UniGNN2
}

print("List 1 (Screenshot 1):")
print(list1_model)
print("\nList 2 (Screenshot 2):")
print(list2_model)
print("\nMapping Dictionary (List 2 -> List 1):")
print(mapping_dict_model)

List 1 (Screenshot 1):
['GCN', 'GAT', 'GIN', 'AST', 'EDGNN', 'UniGNN2', 'CWN', 'CCCN', 'CCXN', 'SCN', 'SCCN', 'SCCNN']

List 2 (Screenshot 2):
['AllSetTransformer', 'CCCN', 'CCXN', 'CWN', 'EDGNN', 'GAT', 'GCN', 'GIN', 'SCCN', 'SCCNNCustom', 'SCN2', 'UniGCNII']

Mapping Dictionary (List 2 -> List 1):
{'AllSetTransformer': 'AST', 'CCCN': 'CCCN', 'CCXN': 'CCXN', 'CWN': 'CWN', 'EDGNN': 'EDGNN', 'GAT': 'GAT', 'GCN': 'GCN', 'GIN': 'GIN', 'SCCN': 'SCCN', 'SCCNNCustom': 'SCCNN', 'SCN2': 'SCN', 'UniGCNII': 'UniGNN2'}


In [54]:
collected_results_time
# Convert nested dictionary to DataFrame
nested_dict = dict(collected_results_time)
result_dict = pd.DataFrame.from_dict(
    {
        (i, j): nested_dict[i][j]
        for i in nested_dict
        for j in nested_dict[i].keys()
    },
    orient="index",
)


result_dict = result_dict.round(2)
result_dict["performance"] = result_dict.apply(
    lambda x: f"{x['mean']} ± {x['std']}", axis=1
)
result_dict = result_dict.drop(["mean", "std"], axis=1)

# Reset multiindex
result_dict = result_dict.reset_index()
# rename columns
result_dict.columns = ["Dataset", "Model", "Average Time per Epoch"]

In [57]:
tb = result_dict.pivot_table(index="Model", columns="Dataset", values="Average Time per Epoch", aggfunc="first")
tb.rename(columns=mapping_dict_data, index=mapping_dict_model, inplace=True)
tb[list1_data].T[list1_model]

Model,GCN,GAT,GIN,AST,EDGNN,UniGNN2,CWN,CCCN,CCXN,SCN,SCCN,SCCNN
Dataset,,,,,,,,,,,,
Cora,0.22 ± 0.0,0.22 ± 0.01,0.22 ± 0.0,0.31 ± 0.0,0.3 ± 0.01,0.34 ± 0.05,0.49 ± 0.01,0.49 ± 0.01,0.5 ± 0.01,0.5 ± 0.01,0.51 ± 0.01,0.51 ± 0.01
Citeseer,0.31 ± 0.01,0.31 ± 0.0,0.31 ± 0.0,0.6 ± 0.01,0.57 ± 0.0,0.6 ± 0.0,0.81 ± 0.01,0.81 ± 0.01,0.82 ± 0.04,0.79 ± 0.01,0.82 ± 0.02,0.81 ± 0.01
Pubmed,0.29 ± 0.0,0.3 ± 0.01,0.29 ± 0.0,0.56 ± 0.01,0.53 ± 0.0,0.88 ± 0.0,1.26 ± 0.02,1.28 ± 0.02,1.3 ± 0.03,1.34 ± 0.01,1.27 ± 0.01,1.32 ± 0.01
MUTAG,0.03 ± 0.0,0.04 ± 0.0,0.03 ± 0.0,0.08 ± 0.0,0.04 ± 0.0,0.04 ± 0.0,0.1 ± 0.0,0.11 ± 0.0,0.09 ± 0.0,0.09 ± 0.0,0.09 ± 0.0,0.09 ± 0.0
PROTEINS,0.05 ± 0.0,0.07 ± 0.0,0.06 ± 0.0,0.16 ± 0.0,0.13 ± 0.0,0.22 ± 0.01,0.37 ± 0.02,0.32 ± 0.01,0.32 ± 0.01,0.42 ± 0.01,0.5 ± 0.01,0.43 ± 0.01
NCI1,0.26 ± 0.01,0.34 ± 0.02,0.27 ± 0.02,0.55 ± 0.02,0.49 ± 0.02,0.77 ± 0.0,1.38 ± 0.03,1.33 ± 0.02,1.25 ± 0.02,1.6 ± 0.03,1.71 ± 0.02,1.65 ± 0.02
NCI109,0.27 ± 0.02,0.33 ± 0.02,0.27 ± 0.02,0.56 ± 0.02,0.49 ± 0.01,0.48 ± 0.01,1.37 ± 0.01,1.33 ± 0.02,1.39 ± 0.04,1.59 ± 0.02,1.7 ± 0.01,1.66 ± 0.03
IMDB-BIN,0.05 ± 0.0,0.06 ± 0.0,0.06 ± 0.0,0.19 ± 0.01,0.09 ± 0.0,0.15 ± 0.0,0.4 ± 0.01,0.44 ± 0.04,0.36 ± 0.02,4.9 ± 1.09,5.33 ± 0.98,5.92 ± 1.19
IMDB-MUL,0.07 ± 0.0,0.09 ± 0.0,0.06 ± 0.0,0.18 ± 0.01,0.13 ± 0.0,0.19 ± 0.01,0.45 ± 0.03,0.48 ± 0.01,0.5 ± 0.02,9.82 ± 1.58,9.32 ± 1.41,9.41 ± 1.36


In [28]:
# result_dict.pivot_table(
#     index="Model", columns="Dataset", values="Average Time per Epoch", aggfunc="first"
# )[['MUTAG', 'NCI1','NCI109','PROTEINS','ZINC']]

### Runtime

In [102]:
df

,AvgTime/train_batch_mean,AvgTime/train_batch_std,AvgTime/train_epoch_mean,AvgTime/train_epoch_std,AvgTime/val_batch_mean,AvgTime/val_batch_std,AvgTime/val_epoch_mean,AvgTime/val_epoch_std,_runtime,_step,...,callbacks.model_checkpoint.train_time_interval,callbacks.model_checkpoint.auto_insert_metric_name,callbacks.model_checkpoint.save_on_train_epoch_end,callbacks.learning_rate_monitor._target_,callbacks.learning_rate_monitor.logging_interval,paths.log_dir,paths.data_dir,paths.root_dir,paths.work_dir,paths.output_dir
0,0.032689,0.002057,0.470203,0.087501,0.015362,0.000041,0.180207,0.006759,68.039344,309,...,None,False,None,lightning.pytorch.callbacks.LearningRateMonitor,epoch,/home/levtel/projects/TopoBench/logs,/home/levtel/projects/TopoBench/datasets,/home/levtel/projects/TopoBench,/home/levtel/projects/TopoBench,/home/levtel/projects/TopoBench/logs/train/mul...
1,0.029907,0.001306,0.452373,0.109531,0.016776,0.000223,0.236805,0.081573,39.657831,188,...,None,False,None,lightning.pytorch.callbacks.LearningRateMonitor,epoch,/home/levtel/projects/TopoBench/logs,/home/levtel/projects/TopoBench/datasets,/home/levtel/projects/TopoBench,/home/levtel/projects/TopoBench,/home/levtel/projects/TopoBench/logs/train/mul...
2,0.031904,0.001412,0.455489,0.080021,0.014566,0.000230,0.173175,0.005567,56.177278,265,...,None,False,None,lightning.pytorch.callbacks.LearningRateMonitor,epoch,/home/levtel/projects/TopoBench/logs,/home/levtel/projects/TopoBench/datasets,/home/levtel/projects/TopoBench,/home/levtel/projects/TopoBench,/home/levtel/projects/TopoBench/logs/train/mul...
3,0.028979,0.001449,0.360252,0.085200,0.017235,0.000039,0.159100,0.005941,39.326062,232,...,None,False,None,lightning.pytorch.callbacks.LearningRateMonitor,epoch,/home/levtel/projects/TopoBench/logs,/home/levtel/projects/TopoBench/datasets,/home/levtel/projects/TopoBench,/home/levtel/projects/TopoBench,/home/levtel/projects/TopoBench/logs/train/mul...
4,0.031983,0.001381,0.456439,0.079781,0.014047,0.000069,0.170974,0.002998,56.192044,265,...,None,False,None,lightning.pytorch.callbacks.LearningRateMonitor,epoch,/home/levtel/projects/TopoBench/logs,/home/levtel/projects/TopoBench/datasets,/home/levtel/projects/TopoBench,/home/levtel/projects/TopoBench,/home/levtel/projects/TopoBench/logs/train/mul...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267,0.209340,0.000706,0.723913,0.015061,0.200128,0.000223,0.284083,0.009031,422.893170,1735,...,None,False,None,lightning.pytorch.callbacks.LearningRateMonitor,epoch,/home/levtel/projects/dev/TopoBench/logs,/home/levtel/projects/dev/TopoBench/datasets,/home/levtel/projects/dev/TopoBench,/home/levtel/projects/dev/TopoBench,/home/levtel/projects/dev/TopoBench/logs/train...
1268,0.209416,0.000566,0.712503,0.013833,0.200060,0.000188,0.275125,0.007907,256.464463,1069,...,None,False,None,lightning.pytorch.callbacks.LearningRateMonitor,epoch,/home/levtel/projects/dev/TopoBench/logs,/home/levtel/projects/dev/TopoBench/datasets,/home/levtel/projects/dev/TopoBench,/home/levtel/projects/dev/TopoBench,/home/levtel/projects/dev/TopoBench/logs/train...
1269,0.209570,0.000832,0.725626,0.072387,0.200108,0.000472,0.276588,0.039548,417.710408,1711,...,None,False,None,lightning.pytorch.callbacks.LearningRateMonitor,epoch,/home/levtel/projects/dev/TopoBench/logs,/home/levtel/projects/dev/TopoBench/datasets,/home/levtel/projects/dev/TopoBench,/home/levtel/projects/dev/TopoBench,/home/levtel/projects/dev/TopoBench/logs/train...
1270,0.209715,0.000781,0.713587,0.017275,0.200134,0.000318,0.276233,0.008388,506.397286,2110,...,None,False,None,lightning.pytorch.callbacks.LearningRateMonitor,epoch,/home/levtel/projects/dev/TopoBench/logs,/home/levtel/projects/dev/TopoBench/datasets,/home/levtel/projects/dev/TopoBench,/home/levtel/projects/dev/TopoBench,/home/levtel/projects/dev/TopoBench/logs/train...


In [58]:
collected_results_time_run
# Convert nested dictionary to DataFrame
nested_dict = dict(collected_results_time_run)
result_dict = pd.DataFrame.from_dict(
    {
        (i, j): nested_dict[i][j]
        for i in nested_dict
        for j in nested_dict[i].keys()
    },
    orient="index",
)


result_dict = result_dict.round(2)
result_dict["performance"] = result_dict.apply(
    lambda x: f"{x['mean']} ± {x['std']}", axis=1
)
result_dict = result_dict.drop(["mean", "std"], axis=1)

# Reset multiindex
result_dict = result_dict.reset_index()
# rename columns
result_dict.columns = ["Dataset", "Model", "Average Training Time"]


In [64]:
# tb = result_dict.pivot_table(index="Model", columns="Dataset", values="Average Training Time", aggfunc="first").T
# tb = tb.rename(columns=mapping_dict_model).T.rename(columns=mapping_dict_data).T
# tb.columns
tb = result_dict.pivot_table(index="Model", columns="Dataset", values="Average Training Time", aggfunc="first")
tb = tb.rename(columns=mapping_dict_data).rename(index=mapping_dict_model)
set(list1_model) == set(tb.T.columns)
set(list1_data) == set(tb.columns)



True

In [93]:
# for every entry take value split with respect to ±, convert to float and take average over the column, add the final row as Average runtime
tb
# assuming tb is your DataFrame

# Step 1: convert '0.123 ± 0.045' → float(0.123)
tb_numeric = tb.applymap(lambda x: float(str(x).split('±')[0].strip()))

# Step 2: compute column-wise mean and std
col_means = tb_numeric.mean(axis=1)
col_stds = tb_numeric.std(axis=1)

# Step 3: create the final row
average_runtime_row = pd.Series(
    [f"{mean:.3f} ± {std:.3f}" for mean, std in zip(col_means, col_stds)],
    index=tb.T.columns,
    name="Average runtime"
)

# Step 4: concatenate the new row
tb_final = pd.concat([tb.T, average_runtime_row])

# Show result
print(tb_final)


                         AST              CCCN              CCXN  \
Bachelor         3.09 ± 0.31       3.01 ± 0.32       3.45 ± 0.29   
Birth            2.18 ± 0.32        3.03 ± 0.3       4.04 ± 0.45   
Cora           40.23 ± 11.82      39.36 ± 6.72      35.74 ± 2.93   
Death            3.13 ± 0.24       4.08 ± 0.51       3.49 ± 0.32   
Election         3.33 ± 0.34       4.15 ± 0.28       3.98 ± 0.42   
IMDB-BIN        21.34 ± 2.96      51.88 ± 12.3      51.2 ± 18.14   
IMDB-MUL        20.89 ± 5.05      73.6 ± 20.07      72.3 ± 13.76   
MUTAG             9.9 ± 3.24      12.04 ± 2.21      10.76 ± 1.89   
Income           3.33 ± 0.35       3.71 ± 0.28       3.42 ± 0.31   
Migration        3.49 ± 0.24       4.61 ± 0.36       4.48 ± 0.79   
NCI1          138.13 ± 46.01   372.36 ± 109.47    244.72 ± 46.09   
NCI109        138.66 ± 26.38     272.3 ± 20.89    225.72 ± 86.57   
PROTEINS        15.81 ± 2.89      41.63 ± 7.23       51.98 ± 8.5   
Pubmed         84.67 ± 13.03    172.63 ± 33.06  

/tmp/ipykernel_1136837/4265865825.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  tb_numeric = tb.applymap(lambda x: float(str(x).split('±')[0].strip()))


In [101]:
print(pd.DataFrame(average_runtime_row).T[list1_model].to_latex())

\begin{tabular}{lllllllllllll}
\toprule
Model & GCN & GAT & GIN & AST & EDGNN & UniGNN2 & CWN & CCCN & CCXN & SCN & SCCN & SCCNN \\
\midrule
Average runtime & 27.237 ± 43.953 & 29.374 ± 42.381 & 26.650 ± 38.897 & 59.081 ± 88.183 & 56.146 ± 85.816 & 102.004 ± 147.619 & 191.180 ± 384.521 & 224.649 ± 479.470 & 188.771 ± 389.498 & 217.482 ± 355.562 & 259.503 ± 444.103 & 420.655 ± 903.885 \\
\bottomrule
\end{tabular}



Model,AST,CCCN,CCXN,CWN,EDGNN,GAT,GCN,GIN,SCCN,SCCNN,SCN,UniGNN2
Dataset,,,,,,,,,,,,
Bachelor,3.09 ± 0.31,3.01 ± 0.32,3.45 ± 0.29,2.99 ± 0.34,2.24 ± 0.38,2.69 ± 0.39,2.28 ± 0.11,1.79 ± 0.28,5.98 ± 1.8,7.84 ± 2.14,5.11 ± 0.67,2.3 ± 0.25
Birth,2.18 ± 0.32,3.03 ± 0.3,4.04 ± 0.45,4.11 ± 0.48,2.2 ± 0.39,2.29 ± 0.33,2.04 ± 0.27,1.93 ± 0.32,10.3 ± 5.81,29.24 ± 16.97,4.99 ± 0.4,2.33 ± 0.29
Cora,40.23 ± 11.82,39.36 ± 6.72,35.74 ± 2.93,48.49 ± 27.58,28.17 ± 3.97,33.51 ± 6.33,19.71 ± 3.18,24.24 ± 7.48,84.04 ± 13.39,65.62 ± 23.49,51.53 ± 14.0,35.28 ± 5.14
Death,3.13 ± 0.24,4.08 ± 0.51,3.49 ± 0.32,4.54 ± 0.55,2.35 ± 0.22,2.1 ± 0.32,2.21 ± 0.3,1.77 ± 0.32,10.31 ± 5.89,8.2 ± 1.82,4.62 ± 0.55,2.66 ± 0.46
Election,3.33 ± 0.34,4.15 ± 0.28,3.98 ± 0.42,4.01 ± 0.39,2.21 ± 0.34,2.66 ± 0.43,2.27 ± 0.29,2.1 ± 0.33,4.23 ± 0.73,9.35 ± 4.52,5.18 ± 0.48,2.97 ± 0.32
IMDB-BIN,21.34 ± 2.96,51.88 ± 12.3,51.2 ± 18.14,61.16 ± 9.33,13.06 ± 4.76,7.48 ± 2.21,8.18 ± 3.33,7.72 ± 1.72,432.23 ± 57.29,515.13 ± 112.53,374.83 ± 132.68,20.75 ± 6.02
IMDB-MUL,20.89 ± 5.05,73.6 ± 20.07,72.3 ± 13.76,39.99 ± 5.18,13.89 ± 3.33,9.79 ± 1.85,10.27 ± 3.58,10.42 ± 3.93,716.44 ± 232.31,895.77 ± 399.41,776.65 ± 147.42,16.74 ± 4.15
MUTAG,9.9 ± 3.24,12.04 ± 2.21,10.76 ± 1.89,10.92 ± 0.96,5.81 ± 1.13,4.16 ± 1.05,3.83 ± 0.89,4.6 ± 0.56,10.71 ± 2.92,14.06 ± 2.51,8.47 ± 2.43,5.5 ± 0.78
Income,3.33 ± 0.35,3.71 ± 0.28,3.42 ± 0.31,3.36 ± 0.28,2.6 ± 0.33,2.66 ± 0.41,2.21 ± 0.18,2.03 ± 0.35,10.51 ± 8.3,5.57 ± 0.7,5.08 ± 0.63,2.94 ± 0.37


In [94]:
tb.T

Model,AST,CCCN,CCXN,CWN,EDGNN,GAT,GCN,GIN,SCCN,SCCNN,SCN,UniGNN2
Dataset,,,,,,,,,,,,
Bachelor,3.09 ± 0.31,3.01 ± 0.32,3.45 ± 0.29,2.99 ± 0.34,2.24 ± 0.38,2.69 ± 0.39,2.28 ± 0.11,1.79 ± 0.28,5.98 ± 1.8,7.84 ± 2.14,5.11 ± 0.67,2.3 ± 0.25
Birth,2.18 ± 0.32,3.03 ± 0.3,4.04 ± 0.45,4.11 ± 0.48,2.2 ± 0.39,2.29 ± 0.33,2.04 ± 0.27,1.93 ± 0.32,10.3 ± 5.81,29.24 ± 16.97,4.99 ± 0.4,2.33 ± 0.29
Cora,40.23 ± 11.82,39.36 ± 6.72,35.74 ± 2.93,48.49 ± 27.58,28.17 ± 3.97,33.51 ± 6.33,19.71 ± 3.18,24.24 ± 7.48,84.04 ± 13.39,65.62 ± 23.49,51.53 ± 14.0,35.28 ± 5.14
Death,3.13 ± 0.24,4.08 ± 0.51,3.49 ± 0.32,4.54 ± 0.55,2.35 ± 0.22,2.1 ± 0.32,2.21 ± 0.3,1.77 ± 0.32,10.31 ± 5.89,8.2 ± 1.82,4.62 ± 0.55,2.66 ± 0.46
Election,3.33 ± 0.34,4.15 ± 0.28,3.98 ± 0.42,4.01 ± 0.39,2.21 ± 0.34,2.66 ± 0.43,2.27 ± 0.29,2.1 ± 0.33,4.23 ± 0.73,9.35 ± 4.52,5.18 ± 0.48,2.97 ± 0.32
IMDB-BIN,21.34 ± 2.96,51.88 ± 12.3,51.2 ± 18.14,61.16 ± 9.33,13.06 ± 4.76,7.48 ± 2.21,8.18 ± 3.33,7.72 ± 1.72,432.23 ± 57.29,515.13 ± 112.53,374.83 ± 132.68,20.75 ± 6.02
IMDB-MUL,20.89 ± 5.05,73.6 ± 20.07,72.3 ± 13.76,39.99 ± 5.18,13.89 ± 3.33,9.79 ± 1.85,10.27 ± 3.58,10.42 ± 3.93,716.44 ± 232.31,895.77 ± 399.41,776.65 ± 147.42,16.74 ± 4.15
MUTAG,9.9 ± 3.24,12.04 ± 2.21,10.76 ± 1.89,10.92 ± 0.96,5.81 ± 1.13,4.16 ± 1.05,3.83 ± 0.89,4.6 ± 0.56,10.71 ± 2.92,14.06 ± 2.51,8.47 ± 2.43,5.5 ± 0.78
Income,3.33 ± 0.35,3.71 ± 0.28,3.42 ± 0.31,3.36 ± 0.28,2.6 ± 0.33,2.66 ± 0.41,2.21 ± 0.18,2.03 ± 0.35,10.51 ± 8.3,5.57 ± 0.7,5.08 ± 0.63,2.94 ± 0.37


In [91]:
tb

Dataset,Bachelor,Birth,Cora,Death,Election,IMDB-BIN,IMDB-MUL,MUTAG,Income,Migration,...,PROTEINS,Pubmed,REDDIT,Unempl,ZINC,Amazon,Citeseer,Minesweeper,Empire,Tolokers
Model,,,,,,,,,,,,,,,,,,,,,
AST,3.09 ± 0.31,2.18 ± 0.32,40.23 ± 11.82,3.13 ± 0.24,3.33 ± 0.34,21.34 ± 2.96,20.89 ± 5.05,9.9 ± 3.24,3.33 ± 0.35,3.49 ± 0.24,...,15.81 ± 2.89,84.67 ± 13.03,72.33 ± 18.33,4.73 ± 0.31,397.77 ± 153.75,61.77 ± 8.17,58.78 ± 2.26,15.79 ± 2.67,82.61 ± 15.26,117.83 ± 23.21
CCCN,3.01 ± 0.32,3.03 ± 0.3,39.36 ± 6.72,4.08 ± 0.51,4.15 ± 0.28,51.88 ± 12.3,73.6 ± 20.07,12.04 ± 2.21,3.71 ± 0.28,4.61 ± 0.36,...,41.63 ± 7.23,172.63 ± 33.06,1653.47 ± 641.13,4.19 ± 0.29,1621.11 ± 141.72,149.34 ± 58.59,48.39 ± 4.57,98.2 ± 15.21,84.54 ± 6.34,NaN
CCXN,3.45 ± 0.29,4.04 ± 0.45,35.74 ± 2.93,3.49 ± 0.32,3.98 ± 0.42,51.2 ± 18.14,72.3 ± 13.76,10.76 ± 1.89,3.42 ± 0.31,4.48 ± 0.79,...,51.98 ± 8.5,212.66 ± 52.26,1435.07 ± 427.2,5.1 ± 0.8,1226.67 ± 317.55,201.99 ± 38.94,53.88 ± 2.81,50.27 ± 14.89,63.27 ± 15.66,NaN
CWN,2.99 ± 0.34,4.11 ± 0.48,48.49 ± 27.58,4.54 ± 0.55,4.01 ± 0.39,61.16 ± 9.33,39.99 ± 5.18,10.92 ± 0.96,3.36 ± 0.28,4.6 ± 0.35,...,53.6 ± 17.7,147.59 ± 28.96,1230.53 ± 270.03,3.97 ± 0.3,1390.21 ± 96.86,239.31 ± 45.79,48.89 ± 4.02,52.12 ± 13.87,67.26 ± 10.32,NaN
EDGNN,2.24 ± 0.38,2.2 ± 0.39,28.17 ± 3.97,2.35 ± 0.22,2.21 ± 0.34,13.06 ± 4.76,13.89 ± 3.33,5.81 ± 1.13,2.6 ± 0.33,2.19 ± 0.3,...,15.15 ± 3.3,87.25 ± 13.88,73.92 ± 26.58,2.17 ± 0.28,357.68 ± 46.95,52.31 ± 4.88,39.59 ± 1.93,20.4 ± 4.38,63.77 ± 13.16,210.78 ± 30.25
GAT,2.69 ± 0.39,2.29 ± 0.33,33.51 ± 6.33,2.1 ± 0.32,2.66 ± 0.43,7.48 ± 2.21,9.79 ± 1.85,4.16 ± 1.05,2.66 ± 0.41,2.17 ± 0.3,...,8.18 ± 2.3,50.2 ± 13.27,26.87 ± 5.02,2.68 ± 0.3,171.15 ± 64.67,25.09 ± 4.41,21.18 ± 2.24,10.64 ± 1.1,27.1 ± 1.89,119.87 ± 42.98
GCN,2.28 ± 0.11,2.04 ± 0.27,19.71 ± 3.18,2.21 ± 0.3,2.27 ± 0.29,8.18 ± 3.33,10.27 ± 3.58,3.83 ± 0.89,2.21 ± 0.18,1.86 ± 0.3,...,8.18 ± 2.47,40.89 ± 8.39,16.17 ± 1.75,2.26 ± 0.35,146.89 ± 27.63,22.12 ± 4.99,22.58 ± 2.48,7.33 ± 1.66,23.38 ± 2.21,163.93 ± 31.44
GIN,1.79 ± 0.28,1.93 ± 0.32,24.24 ± 7.48,1.77 ± 0.32,2.1 ± 0.33,7.72 ± 1.72,10.42 ± 3.93,4.6 ± 0.56,2.03 ± 0.35,1.87 ± 0.31,...,8.88 ± 2.34,59.91 ± 10.56,28.75 ± 7.64,2.02 ± 0.31,168.43 ± 107.1,15.25 ± 2.73,24.65 ± 2.58,8.32 ± 2.51,22.36 ± 4.9,77.75 ± 26.74
SCCN,5.98 ± 1.8,10.3 ± 5.81,84.04 ± 13.39,10.31 ± 5.89,4.23 ± 0.73,432.23 ± 57.29,716.44 ± 232.31,10.71 ± 2.92,10.51 ± 8.3,4.15 ± 0.76,...,70.06 ± 15.91,134.82 ± 34.81,1622.17 ± 667.71,4.17 ± 0.73,1226.19 ± 1686.57,NaN,65.53 ± 13.34,28.76 ± 3.74,94.94 ± 27.26,NaN


In [89]:
average_runtime_row

Model
AST          59.081 ± 88.183
CCCN       224.649 ± 479.470
CCXN       188.771 ± 389.498
CWN        191.180 ± 384.521
EDGNN        56.146 ± 85.816
GAT          29.374 ± 42.381
GCN          27.237 ± 43.953
GIN          26.650 ± 38.897
SCCN       259.503 ± 444.103
SCCNN      420.655 ± 903.885
SCN        217.482 ± 355.562
UniGNN2    102.004 ± 147.619
Name: Average runtime, dtype: object

In [86]:
pd.DataFrame(average_runtime_row)

,Average runtime
Dataset,
Bachelor,3.564 ± 1.816
Birth,5.723 ± 7.766
Cora,42.160 ± 18.161
Death,4.122 ± 2.616
Election,3.870 ± 1.977
IMDB-BIN,130.413 ± 190.425
IMDB-MUL,221.396 ± 349.564
MUTAG,8.397 ± 3.480
Income,3.952 ± 2.320


In [68]:
print(tb[list1_data].T[list1_model].to_latex())

\begin{tabular}{lllllllllllll}
\toprule
Model & GCN & GAT & GIN & AST & EDGNN & UniGNN2 & CWN & CCCN & CCXN & SCN & SCCN & SCCNN \\
Dataset &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
Cora & 19.71 ± 3.18 & 33.51 ± 6.33 & 24.24 ± 7.48 & 40.23 ± 11.82 & 28.17 ± 3.97 & 35.28 ± 5.14 & 48.49 ± 27.58 & 39.36 ± 6.72 & 35.74 ± 2.93 & 51.53 ± 14.0 & 84.04 ± 13.39 & 65.62 ± 23.49 \\
Citeseer & 22.58 ± 2.48 & 21.18 ± 2.24 & 24.65 ± 2.58 & 58.78 ± 2.26 & 39.59 ± 1.93 & 41.58 ± 3.55 & 48.89 ± 4.02 & 48.39 ± 4.57 & 53.88 ± 2.81 & 58.85 ± 18.23 & 65.53 ± 13.34 & 80.6 ± 36.33 \\
Pubmed & 40.89 ± 8.39 & 50.2 ± 13.27 & 59.91 ± 10.56 & 84.67 ± 13.03 & 87.25 ± 13.88 & 160.37 ± 31.88 & 147.59 ± 28.96 & 172.63 ± 33.06 & 212.66 ± 52.26 & 151.0 ± 22.55 & 134.82 ± 34.81 & 171.92 ± 48.25 \\
MUTAG & 3.83 ± 0.89 & 4.16 ± 1.05 & 4.6 ± 0.56 & 9.9 ± 3.24 & 5.81 ± 1.13 & 5.5 ± 0.78 & 10.92 ± 0.96 & 12.04 ± 2.21 & 10.76 ± 1.89 & 8.47 ± 2.43 & 10.71 ± 2.92 & 14.06 ± 2.51 \\
PROTEINS & 8.18 ± 2.47 & 8.18 ± 2.3 & 8.8

In [25]:
result_dict.pivot_table(
    index="Model", columns="Dataset", values="Average Training Time", aggfunc="first"
)[['MUTAG', 'NCI1','NCI109','PROTEINS','ZINC']]

Dataset,MUTAG,NCI1,NCI109,PROTEINS,ZINC
Model,,,,,
AllSetTransformer,9.9 ± 3.24,138.13 ± 46.01,138.66 ± 26.38,15.81 ± 2.89,397.77 ± 153.75
CCCN,12.04 ± 2.21,372.36 ± 109.47,272.3 ± 20.89,41.63 ± 7.23,1621.11 ± 141.72
CCXN,10.76 ± 1.89,244.72 ± 46.09,225.72 ± 86.57,51.98 ± 8.5,1226.67 ± 317.55
CWN,10.92 ± 0.96,302.34 ± 63.44,294.79 ± 46.27,53.6 ± 17.7,1390.21 ± 96.86
EDGNN,5.81 ± 1.13,110.86 ± 27.75,126.61 ± 45.53,15.15 ± 3.3,357.68 ± 46.95
GAT,4.16 ± 1.05,57.32 ± 17.49,56.44 ± 9.05,8.18 ± 2.3,171.15 ± 64.67
GCN,3.83 ± 0.89,53.23 ± 19.67,37.4 ± 8.63,8.18 ± 2.47,146.89 ± 27.63
GIN,4.6 ± 0.56,61.2 ± 23.97,50.32 ± 7.98,8.88 ± 2.34,168.43 ± 107.1
SCCN,10.71 ± 2.92,332.76 ± 51.89,321.76 ± 55.92,70.06 ± 15.91,1226.19 ± 1686.57
